In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import os

os.environ["PICSELLIA_SDK_CUSTOM_LOGGING"] = "True"
os.environ["PICSELLIA_SDK_DOWNLOAD_BAR_MODE"] = "2"
logging.getLogger("picsellia").setLevel(logging.INFO)
os.environ["api_token"] = "6ccfdfbfc3e1393408b684e4df5c0c89f292fe3b"
os.environ["experiment_id"] = "018cb020-2895-73bf-9cdb-471d85f2cff4"
os.environ["organization_id"] = "2856a16a-8d44-4c11-b24f-692cc2276afa"


In [3]:
import picsellia
from picsellia import Experiment


def get_experiment() -> Experiment:
    if "api_token" not in os.environ:
        raise Exception("You must set an api_token to run this image")
    api_token = os.environ["api_token"]

    if "host" not in os.environ:
        host = "https://app.picsellia.com"
    else:
        host = os.environ["host"]

    if "organization_id" not in os.environ:
        organization_id = None
    else:
        organization_id = os.environ["organization_id"]

    client = picsellia.Client(
        api_token=api_token, host=host, organization_id=organization_id
    )

    if "experiment_id" in os.environ:
        experiment_id = os.environ["experiment_id"]

        experiment = client.get_experiment_by_id(experiment_id)
    else:
        raise Exception(
            "You must set the experiment_id"
        )
    return experiment


experiment = get_experiment()


In [4]:
from pycocotools.coco import COCO
import json
from picsellia.exceptions import ResourceNotFoundError
from experiment.utils import create_yolo_detection_label

experiment.download_artifacts(with_tree=True)
current_dir = os.path.join(os.getcwd(), experiment.base_dir)
base_imgdir = experiment.png_dir

parameters = experiment.get_log(name="parameters").data
attached_datasets = experiment.list_attached_dataset_versions()

if len(attached_datasets) == 3:
    try:
        train_ds = experiment.get_dataset(name="train")
    except Exception:
        raise ResourceNotFoundError("Found 3 attached datasets, but can't find any 'train' dataset.\n \
                                            expecting 'train', 'test', ('val' or 'eval')")
    try:
        test_ds = experiment.get_dataset(name="test")
    except Exception:
        raise ResourceNotFoundError("Found 3 attached datasets, but can't find any 'test' dataset.\n \
                                            expecting 'train', 'test', ('val' or 'eval')")
    try:
        val_ds = experiment.get_dataset(name="val")
    except Exception:
        try:
            val_ds = experiment.get_dataset(name="eval")
        except Exception:
            raise ResourceNotFoundError("Found 3 attached datasets, but can't find any 'eval' dataset.\n \
                                                expecting 'train', 'test', ('val' or 'eval')")

    labels = train_ds.list_labels()
    label_names = [label.name for label in labels]
    labelmap = {str(i): label.name for i, label in enumerate(labels)}

    # for data_type, dataset in {
    #     "train": train_ds,
    #     "val": val_ds,
    #     "test": test_ds,
    # }.items():
    #     coco_annotation = dataset.build_coco_file_locally(enforced_ordered_categories=label_names)
    #     annotations_dict = coco_annotation.dict()
    #     annotations_path = os.path.join(base_imgdir, f"{data_type}_annotations.json")
    # 
    #     with open(annotations_path, 'w') as f:
    #         f.write(json.dumps(annotations_dict))
    # 
    #     annotations_coco = COCO(annotations_path)
    # 
    #     dataset.list_assets().download(
    #         target_path=os.path.join(base_imgdir, data_type, "images"), max_workers=8
    #     )

else:
    dataset = experiment.list_attached_dataset_versions()[0]
    coco_annotation = dataset.build_coco_file_locally()
    annotations_dict = coco_annotation.dict()
    annotations_path = "annotations.json"
    with open(annotations_path, 'w') as f:
        f.write(json.dumps(annotations_dict))
    annotations_coco = COCO(annotations_path)

    labels = dataset.list_labels()
    label_names = [label.name for label in labels]
    labelmap = {str(i): label.name for i, label in enumerate(labels)}

    prop = (
        0.7
        if not "prop_train_split" in parameters.keys()
        else parameters["prop_train_split"]
    )

    train_assets, test_assets, val_assets = dataset.train_test_val_split(
        ratios=[prop, (1. - prop) / 2, (1. - prop) / 2]
    )

    for data_type, assets in {
        "train": train_assets,
        "val": val_assets,
        "test": test_assets,
    }.items():
        assets.download(
            target_path=os.path.join(base_imgdir, data_type, "images"), max_workers=8
        )

        create_yolo_detection_label(
            experiment, data_type, annotations_dict, annotations_coco, label_names
        )

experiment.log("labelmap", labelmap, "labelmap", replace=True)
cwd = os.getcwd()

In [5]:
if os.path.isdir(f"{experiment.png_dir}/train"):
    os.rename(f"{experiment.png_dir}/train",
              f"{experiment.png_dir}/train2017")

if os.path.isdir(f"{experiment.png_dir}/test"):
    os.rename(f"{experiment.png_dir}/test",
              f"{experiment.png_dir}/test2017")

if os.path.isdir(f"{experiment.png_dir}/val"):
    os.rename(f"{experiment.png_dir}/val",
              f"{experiment.png_dir}/val2017")



In [6]:
parameters = experiment.get_log("parameters").data

if "architecture" not in parameters:
    raise AssertionError("The YoloX cannot be created since the model parameter \"architecture\" is missing.")

model_architecture = parameters["architecture"]
dataset_base_dir_path = experiment.base_dir
dataset_train_annotation_file_path = f"{experiment.png_dir}/train_annotations.json"
dataset_test_annotation_file_path = f"{experiment.png_dir}/test_annotations.json"
dataset_val_annotation_file_path = f"{experiment.png_dir}/val_annotations.json"
model_latest_checkpoint_path = f"{experiment.checkpoint_dir}/{model_architecture}.pth"

if not os.path.isfile(model_latest_checkpoint_path):
    model_latest_checkpoint_path = None

learning_rate = parameters.get("learning_rate", 0.01 / 64)
batch_size = parameters.get("batch_size", 8)
epochs = parameters.get("epochs", 100)
image_size = int(parameters.get("image_size", 640))


In [7]:
from YOLOX.tools.train import main, make_parser
from YOLOX.yolox.exp import check_exp_value
from YOLOX.yolox.utils import configure_module, get_num_devices
from YOLOX.yolox.exp.build import get_exp_by_name

# 1 - Args
configure_module()
args = make_parser().parse_args()

args.name = model_architecture

args.data_dir = experiment.png_dir
args.train_ann = dataset_train_annotation_file_path
args.test_ann = dataset_test_annotation_file_path
args.val_ann = dataset_val_annotation_file_path
args.ckpt = model_latest_checkpoint_path

args.num_classes = len(labelmap)
args.learning_rate = learning_rate
args.batch_size = batch_size
args.epochs = epochs
args.image_size = (image_size, image_size)

# 2 - Get model architecture
exp = get_exp_by_name(args)
exp.merge(args.opts)
check_exp_value(exp)

if not args.experiment_name:
    args.experiment_name = exp.exp_name

num_gpu = get_num_devices() if args.devices is None else args.devices
assert num_gpu <= get_num_devices()

# 3 - Launch training
main(exp, args)

2023-12-28 16:36:16 | INFO     | YOLOX.yolox.core.trainer:129 - args: Namespace(experiment_name='yolox_s', name='yolox_s', dist_backend='nccl', dist_url=None, batch_size=4, devices=None, exp_file='/home/alexis/.local/share/jupyter/runtime/kernel-2200a8a3-1d09-4d00-b1f4-87aec3266758.json', resume=False, ckpt='test-yolox_s-1/checkpoint/yolox_s.pth', start_epoch=None, num_machines=1, machine_rank=0, fp16=False, cache=None, occupy=False, logger='tensorboard', opts=[], data_dir='test-yolox_s-1/images', train_ann='test-yolox_s-1/images/train_annotations.json', test_ann='test-yolox_s-1/images/test_annotations.json', val_ann='test-yolox_s-1/images/val_annotations.json', num_classes=3, learning_rate=0.0001, epochs=1, image_size=(640, 640))
2023-12-28 16:36:16 | INFO     | YOLOX.yolox.core.trainer:130 - exp value:
╒═══════════════════╤════════════════════════════════════════════════╕
│ keys              │ values                                         │
╞═══════════════════╪═════════════════════

 98%|#########7| 79/81 [00:04<00:00, 17.03it/s]

2023-12-28 16:38:47 | INFO     | YOLOX.yolox.evaluators.coco_evaluator:282 - Evaluate in main process...


100%|##########| 81/81 [00:05<00:00, 13.99it/s]

2023-12-28 16:38:47 | INFO     | YOLOX.yolox.evaluators.coco_evaluator:315 - Loading and preparing results...
2023-12-28 16:38:47 | INFO     | YOLOX.yolox.evaluators.coco_evaluator:315 - DONE (t=0.07s)
2023-12-28 16:38:47 | INFO     | pycocotools.coco:366 - creating index...
2023-12-28 16:38:47 | INFO     | pycocotools.coco:366 - index created!
2023-12-28 16:38:48 | INFO     | YOLOX.yolox.layers.jit_ops:119 - Load fast_cocoeval op in 0.405s.



/home/alexis/Documents/Projects/picsellia-training-engine/yolox-detection/YOLOX/yolox/evaluators/coco_evaluator.py:210: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  statistics = torch.cuda.FloatTensor([inference_time, nms_time, n_samples])
Using /home/alexis/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
Emitting ninja build file /home/alexis/.cache/torch_extensions/py310_cu121/fast_cocoeval/build.ninja...
Building extension module fast_cocoeval...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.


2023-12-28 16:38:48 | INFO     | YOLOX.yolox.core.trainer:358 - 
Average forward time: 9.80 ms, Average NMS time: 1.46 ms, Average inference time: 11.25 ms
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.386
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.582
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.462
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.378
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.575
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.200
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.610
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.612
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.600
 Average Recall     (AR) @[ IoU=0.50:0.95 |

In [ ]:


training_pipeline = YoloxDetectionTrainer()
training_pipeline.prepare_data_for_training()

training_pipeline.train()
training_pipeline.eval()
